# Принцип работы системы рекомендаций приложения LinkHub

В данном ноутбуке приведен прицип проверки фильмов на подходящесть пользователю. В реальном проекте загрузка данных происходит  не из CSV файлов, а из базы данных. Для наглядности, в данном примере используются отдельные CSV файлы с подготовленными данными

## Порядок действий:

### 1. Загрузка в Pandas DataFrame проектов, с которыми взаимодействовал пользователь, а именно:
- проекты, в разработке которых принимал участие пользователь: как создатель или редактор 
- проекты, которым пользователь поставил отметку "Нравится"
- проекты, которые пользователь добавил в раздел "Сохраненные"
- проекты, страницы которых пользователь посещал больше 25 раз в течении последних 2 дней (в расчет берется посещение страницы описания проекта, разделов и источников)

In [31]:
import pandas as pd
user_df = pd.read_csv('C:\\Users\\Home PC\\Desktop\\html-templates\\user.csv')
user_df

,main_admin,source_amount,links_percentage,image_percentage,video_percentage,document_percentage,text_percentage,stars_amount,themes
0,1,2,0.50,0.5,1.0,0.5,0.0,10,['SQL']
1,1,10,1.00,0.0,0.0,1.0,0.0,5,"['Python', 'Математика']"
2,1,3,0.50,0.0,0.0,0.0,0.0,3,"['Python', 'Программирование']"
3,1,22,0.75,1.0,0.5,0.0,0.0,0,"['Python', 'Программирование']"
4,1,7,0.30,0.0,1.0,0.0,0.5,0,"['Python', 'Образование']"
5,1,5,1.00,0.0,0.0,0.5,0.0,2,"['Python', 'Машинное обучение']"


### 2. Загрузка в Pandas DataFrame проектов, заинтересованность в которых необходимо определить

In [32]:
import pandas as pd
random_projects_df = pd.read_csv('C:\\Users\\Home PC\\Desktop\\html-templates\\random.csv')
random_projects_df

,id,main_admin,source_amount,links_percentage,image_percentage,video_percentage,document_percentage,text_percentage,stars_amount,themes
0,10,2,4,0.50,0.5,1.0,0.0,0.0,10,"['Python', 'Собеседование']"
1,2,3,10,0.20,0.1,0.0,0.0,0.5,3,"['Python', 'Программирование']"
2,4,2,4,0.75,0.5,0.2,0.0,0.3,12,['Машинное обучение']
3,8,2,10,0.20,0.4,1.0,1.0,1.0,2,['Программирование']


### 3. Удалить столбец id
Удалить из датафрэйма столбец id и сохранить его отдельно. Он понадобится в дальнейшем для интерпретации результата и ранжировании проектов

In [33]:
ids = random_projects_df.id.values
random_projects_df.drop(['id'], axis=1, inplace=True)
random_projects_df

,main_admin,source_amount,links_percentage,image_percentage,video_percentage,document_percentage,text_percentage,stars_amount,themes
0,2,4,0.50,0.5,1.0,0.0,0.0,10,"['Python', 'Собеседование']"
1,3,10,0.20,0.1,0.0,0.0,0.5,3,"['Python', 'Программирование']"
2,2,4,0.75,0.5,0.2,0.0,0.3,12,['Машинное обучение']
3,2,10,0.20,0.4,1.0,1.0,1.0,2,['Программирование']


Замечение: для реализации в JupyterNotebook необходимо привести темы к типу List, т.к. в CSV файле информация о темах проекта сохраняется как String значение.

In [34]:
user_df['themes'] = user_df['themes'].apply(lambda x:list(map(lambda y: y.strip()[1:-1],x[1:-1].split(","))))

In [35]:
random_projects_df['themes'] = random_projects_df['themes'].apply(lambda x:list(map(lambda y: y.strip()[1:-1],x[1:-1].split(","))))

### 4. Бинаризация тем: 
Multiple Hot Encoding - преобразовать список тем в n бинарных признаков.

Пример: список ['Программирование', 'SQL', 'Python'] будет преобразован в отдельные признаки 'Программирование', 'SQL', 'Python', где значения:

1 - у данного проекта есть эта тема

0 - у данного проекта нет этой темы

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [37]:
user_df = user_df.join(pd.DataFrame(mlb.fit_transform(user_df.pop('themes')),
                                                  columns=mlb.classes_,
                                                  index = df.index))

In [38]:
random_projects_df = random_projects_df.join(pd.DataFrame(mlb.fit_transform(random_projects_df.pop('themes')),
                                                  columns=mlb.classes_,
                                                  index = random_projects_df.index))

### 5. Расчет корреляций
Расчитывается корреляция каждого проекта из датафрэйма случайных проектов с каждым проектом из датафрэйма пользователя, после чего происходит агрерация для одного проекта и рассчитывается среднее арифметическое корреляций случайного проекта с проектами пользователя. В результате каждый случайный проект будет инициализирован средней корреляцией.

In [39]:
lst = []
for k in range(random_projects_df.shape[0]):
    s = sum(random_projects_df.iloc[k].corr(user_df.iloc[i]) for i in range(user_df.shape[0])) / user_df.shape[0]
    lst.append(s)
lst

[0.5771509520909066,
 0.8094105130912327,
 0.5328958651814661,
 0.7765725874584296]

In [40]:
random_projects_df['avg_corr'] = lst

In [41]:
random_projects_df

,main_admin,source_amount,links_percentage,image_percentage,video_percentage,document_percentage,text_percentage,stars_amount,Python,Машинное обучение,Программирование,Собеседование,avg_corr
0,2,4,0.50,0.5,1.0,0.0,0.0,10,1,0,0,1,0.577151
1,3,10,0.20,0.1,0.0,0.0,0.5,3,1,0,1,0,0.809411
2,2,4,0.75,0.5,0.2,0.0,0.3,12,0,1,0,0,0.532896
3,2,10,0.20,0.4,1.0,1.0,1.0,2,0,0,1,0,0.776573


### 6. Вернуть столбец id в датафрэйм 

In [42]:
random_projects_df['id'] = ids
random_projects_df

,main_admin,source_amount,links_percentage,image_percentage,video_percentage,document_percentage,text_percentage,stars_amount,Python,Машинное обучение,Программирование,Собеседование,avg_corr,id
0,2,4,0.50,0.5,1.0,0.0,0.0,10,1,0,0,1,0.577151,10
1,3,10,0.20,0.1,0.0,0.0,0.5,3,1,0,1,0,0.809411,2
2,2,4,0.75,0.5,0.2,0.0,0.3,12,0,1,0,0,0.532896,4
3,2,10,0.20,0.4,1.0,1.0,1.0,2,0,0,1,0,0.776573,8


### 7. Ранжировать проекты по средней корреляции и выбрать определенное количество проектов с наибольшим значением.
Возвращаемый результат: список проектов, которые больше всего похожи. В данном примере выбирается 2 наиболее похожим проекта. 

In [43]:
list(random_projects_df.nlargest(2, 'avg_corr').id.values)

[2, 8]